In [1]:
from dlcliche.notebook import *
from dlcliche.utils import *

In [2]:
import torchvision
import logging
from sklearn.metrics import average_precision_score


logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [3]:
EVEN, ODD = 10, 11


class MultiLabelMNIST(torchvision.datasets.MNIST):
    def __init__(self, folder, train, download=False):
        super().__init__(folder, train=train, download=download)

    def __getitem__(self, index):
        x, y = super().__getitem__(index)
        label = [0,0,0,0,0, 0,0,0,0,0, 0,0]
        # multi label
        label[y] = 1 # one of class 0-9
        label[ODD if y % 2 == 1 else EVEN] = 1 # odd or even
        return x, label


org_train = MultiLabelMNIST('data', train=True, download=True)
org_test = MultiLabelMNIST('data', train=False)
X = org_train.data.view(-1, 28*28).numpy()
y = [org_train[i][1] for i in range(len(org_train))]
test_X = org_test.data.view(-1, 28*28).numpy()
test_y = [org_test[i][1] for i in range(len(org_test))]

## TorchMLPClassifier

In [4]:
from MLP.torch_mlp_clf import TorchMLPClassifier

clf = TorchMLPClassifier(debug=True)
clf.fit(X, y)
print(clf.score(test_X, test_y))

# mAP
preds = clf.predict(test_X, multi_label_n_class=12)
print('mAP:', average_precision_score(test_y, preds))

for i in range(preds.shape[1]):
    print(i, 'precision:', average_precision_score(np.array(test_y)[:, i], preds[:, i]))

mAP BCEWithLogitsLoss() 12 <class 'torch.Tensor'>


INFO:MLP.torch_mlp_clf:epoch 0001/200: lr: 0.0010000: loss=23.627385 val_mAP=0.9740124 val_loss=11.4058819
INFO:MLP.torch_mlp_clf:epoch 0002/200: lr: 0.0010000: loss=9.406010 val_mAP=0.9819052 val_loss=9.1678877
INFO:MLP.torch_mlp_clf:epoch 0003/200: lr: 0.0010000: loss=7.161456 val_mAP=0.9853427 val_loss=8.1329918
INFO:MLP.torch_mlp_clf:epoch 0004/200: lr: 0.0010000: loss=5.893633 val_mAP=0.9867779 val_loss=7.6424270
INFO:MLP.torch_mlp_clf:epoch 0005/200: lr: 0.0010000: loss=5.006749 val_mAP=0.9889223 val_loss=7.0023370
INFO:MLP.torch_mlp_clf:epoch 0006/200: lr: 0.0010000: loss=4.284182 val_mAP=0.9889400 val_loss=6.9350314
INFO:MLP.torch_mlp_clf:epoch 0007/200: lr: 0.0010000: loss=3.801258 val_mAP=0.9893770 val_loss=6.8715057
INFO:MLP.torch_mlp_clf:epoch 0008/200: lr: 0.0010000: loss=3.342514 val_mAP=0.9889166 val_loss=7.0793004
INFO:MLP.torch_mlp_clf:epoch 0009/200: lr: 0.0010000: loss=2.976077 val_mAP=0.9890925 val_loss=7.2710242
INFO:MLP.torch_mlp_clf:epoch 0010/200: lr: 0.0010000:

0.9910060679663474
mAP: 0.9910060679663474
0 precision: 0.9974057239126866
1 precision: 0.9980386957979234
2 precision: 0.990885893298892
3 precision: 0.9891309842491051
4 precision: 0.9937799826417528
5 precision: 0.9844585935491698
6 precision: 0.9938719856398166
7 precision: 0.9853184030533945
8 precision: 0.9820725280985699
9 precision: 0.9872555320460162
10 precision: 0.9966258949123221
11 precision: 0.993228598396521


## Normal MLPClassifier

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
test_X = scaler.transform(test_X)

clf = MLPClassifier()
clf.fit(X, y)
clf.score(test_X, test_y)

0.9449

In [5]:
# mAP
preds = clf.predict(test_X)
print('mAP:', average_precision_score(test_y, preds))

for i in range(preds.shape[1]):
    print(i, 'precision:', average_precision_score(np.array(test_y)[:, i], preds[:, i]))

mAP: 0.9474026367800934
0 precision: 0.9628405304861388
1 precision: 0.9777427576937121
2 precision: 0.9368970334509699
3 precision: 0.9338057461457567
4 precision: 0.93732925778772
5 precision: 0.9328777249016199
6 precision: 0.9509429714683367
7 precision: 0.9358512465490314
8 precision: 0.9264177220742398
9 precision: 0.9219176843280218
10 precision: 0.9751203970268995
11 precision: 0.9770885694486754
